# Funcionarios Públicos

El presente notobook permite descargar, limpiar y almacenar los datos provenietes del [repositorio de datos abiertos de personal](https://www.portaltransparencia.cl/PortalPdT/web/guest/opendata#_48_INSTANCE_GI66ozEZ7DNy_=dataset%2Ftransparencia-activa-publicada-en-el-portal) del Consejo Para La Transparencia (CPLT).
Para ejecutar este notebook, se tienen los siguiente prerequisitos:

- Tener instalado el gestor de base de datos MySQL en algún servidor accesible por el usuario con permisos de escritura.
- Tener instalado Anaconda Python y creado un ambiente que contenga las siguientes librerías: pandas, numpy, time, date, datetime, functools, urllib, json, mysql.connector, sqlalchemy


### Descarga de datos desde el Portal de Transparencia
⚠️ **La ejecución de este notebook requiere de al menos 32GB de RAM**, de preferencia 64 GB de RAM par evitar uso de SWAP memeroy, especialmente si utilizas discos HDD y no SSD

In [ ]:
# librerias necesarias
import pandas as pd

# Descarga de las cuatro bases (planta, contrata, honorarios, cod trabajo)
#Planta
planta = pd.read_csv('http://www.cplt.cl/transparencia_activa/datoabierto/archivos/TA_PersonalPlanta.csv', encoding = 'cp1252', sep = ';')
plata.to_csv('G:/temp/planta.csv')
planta['Regimen'] = 'Planta'
#Contrata
contrata = pd.read_csv('http://www.cplt.cl/transparencia_activa/datoabierto/archivos/TA_PersonalContrata.csv', encoding = 'cp1252', sep = ';')
contrata['Regimen'] = 'Contrata'
#Honorarios
honorarios = pd.read_csv('http://www.cplt.cl/transparencia_activa/datoabierto/archivos/TA_PersonalContratohonorarios.csv', encoding = 'cp1252', sep = ';')
honorarios['Regimen'] = 'Honorarios'
honorarios = honorarios.rename(columns={'remuneracionbruta': 'remuneracionbruta_mensual'})
#Código del Trabajo
codTrabajo = pd.read_csv('http://www.cplt.cl/transparencia_activa/datoabierto/archivos/TA_PersonalCodigotrabajo.csv', encoding = 'cp1252', sep = ';')
codTrabajo['Regimen'] = 'Código del Trabajo'

planta.to_csv('G:/temp/planta.csv')
contrata.to_csv('G:/temp/contrata.csv')
honorarios.to_csv('G:/temp/honorarios.csv')
codTrabajo.to_csv('G:/temp/codTrabajo.csv')

In [5]:
import pandas as pd

# Descarga de las cuatro bases (planta, contrata, honorarios, cod trabajo)
#Planta
planta = pd.read_csv('G:/temp/TA_PersonalPlanta.csv', encoding = 'cp1252', sep = ';')
planta['Regimen'] = 'Planta'
#Contrata
contrata = pd.read_csv('G:/temp/TA_PersonalContrata.csv', encoding = 'cp1252', sep = ';')
contrata['Regimen'] = 'Contrata'
#Honorarios
honorarios = pd.read_csv('G:/temp/TA_PersonalContratohonorarios.csv', encoding = 'cp1252', sep = ';')
honorarios['Regimen'] = 'Honorarios'
honorarios = honorarios.rename(columns={'remuneracionbruta': 'remuneracionbruta_mensual'})
#Código del Trabajo
codTrabajo = pd.read_csv('G:/temp/TA_PersonalCodigotrabajo.csv', encoding = 'cp1252', sep = ';')
codTrabajo['Regimen'] = 'Código del Trabajo'

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
#Unimos los datasets
funHist = pd.concat([planta, contrata, codTrabajo, honorarios], sort = True)


#### **Limpieza de cada campo**

In [23]:
# eliminamos campos innecesarios
funHist = funHist.drop(columns = {'camino', 'enlace', 'observaciones', 'remuliquida_mensual'})

KeyError: "['enlace' 'camino' 'remuliquida_mensual' 'observaciones'] not found in axis"

In [22]:
#Limpieza de las horas extras
#Diurnas
funHist['montoHExDiu'] = pd.DataFrame(funHist['diurnas'].str.split(' ', expand = True))[0] #primero monto
funHist['montoHExDiu'] = funHist['montoHExDiu'].str.replace(',', '').str.replace('No', '0').str.replace('$', '') #eliminamos caracteres inncecesarios y dejamos solo números
funHist['montoHExDiu'] = pd.to_numeric(funHist['montoHExDiu'], errors = 'coerce') # convertimos a números
funHist['montoHExDiu'] = funHist['montoHExDiu'].fillna(0.0) # imputamos missing

funHist['cantHExDiu'] = pd.DataFrame(funHist['diurnas'].str.split(' ', expand = True))[1] #segundo cantidad
funHist['cantHExDiu'] = funHist['cantHExDiu'].str.replace('tiene', '0').str.replace('hrs', '') #eliminamos caracteres inncecesarios y dejamos solo números
funHist['cantHExDiu'] = pd.to_numeric(funHist['cantHExDiu'], errors = 'coerce') # convertimos a números
funHist['cantHExDiu'] = funHist['cantHExDiu'].fillna(0.0) # imputamos missing

#Nocturnas
funHist['montoHExNoc'] = pd.DataFrame(funHist['nocturnas'].str.split(' ', expand = True))[0] #primero monto
funHist['montoHExNoc'] = funHist['montoHExNoc'].str.replace(',', '').str.replace('No', '0').str.replace('$', '') #eliminamos caracteres inncecesarios y dejamos solo números
funHist['montoHExNoc'] = pd.to_numeric(funHist['montoHExNoc'], errors = 'coerce') # convertimos a números
funHist['montoHExNoc'] = funHist['montoHExNoc'].fillna(0.0) # imputamos missing

funHist['cantHExNoc'] = pd.DataFrame(funHist['nocturnas'].str.split(' ', expand = True))[1] #segundo cantidad
funHist['cantHExNoc'] = funHist['cantHExNoc'].str.replace('tiene', '0').str.replace('hrs', '') #eliminamos caracteres inncecesarios y dejamos solo números
funHist['cantHExNoc'] = pd.to_numeric(funHist['cantHExNoc'], errors = 'coerce') # convertimos a números
funHist['cantHExNoc'] = funHist['cantHExNoc'].fillna(0.0) # imputamos missing

#Festivas
funHist['montoHExFes'] = pd.DataFrame(funHist['festivas'].str.split(' ', expand = True))[0] #primero monto
funHist['montoHExFes'] = funHist['montoHExFes'].str.replace(',', '').str.replace('No', '0').str.replace('$', '') #eliminamos caracteres inncecesarios y dejamos solo números
funHist['montoHExFes'] = pd.to_numeric(funHist['montoHExFes'], errors = 'coerce') # convertimos a números
funHist['montoHExFes'] = funHist['montoHExFes'].fillna(0.0) # imputamos missing

funHist['cantHExFes'] = pd.DataFrame(funHist['festivas'].str.split(' ', expand = True))[1] #segundo cantidad
funHist['cantHExFes'] = funHist['cantHExFes'].str.replace('tiene', '0').str.replace('hrs', '') #eliminamos caracteres inncecesarios y dejamos solo números
funHist['cantHExFes'] = pd.to_numeric(funHist['cantHExFes'], errors = 'coerce') # convertimos a números
funHist['cantHExFes'] = funHist['cantHExFes'].fillna(0.0) # imputamos missing

In [24]:
#Limpieza de fechas
funHist['fecha_ingreso'] = funHist['fecha_ingreso'].str.replace(' 00:00:00.000', '')
funHist['fecha_ingreso'] = pd.to_datetime(funHist.fecha_ingreso).dt.strftime('%d/%m/%Y')

funHist['fecha_ingreso'] = funHist['fecha_publicacion'].str.replace(' 00:00:00.000', '')
funHist['fecha_ingreso'] = pd.to_datetime(funHist.fecha_ingreso).dt.strftime('%d/%m/%Y')

funHist['fecha_termino'] = funHist['fecha_termino'].str.replace(r'Indefinido|Sin fecha término','').str.replace(r'31\/12\/(([2-9][0-9][3-9][0-9])|([3-9][0-9][0-9][0-9])|([2-9][1-9][0-9][0-9]))', '').fillna('')
funHist['fecha_termino'] = pd.to_datetime(funHist['fecha_termino'], errors = 'coerce').dt.strftime('%d/%m/%Y')

In [25]:
#limpieza de Tipo Cargo
funHist['Tipo cargo'] = funHist['Tipo cargo'].str.title()

#Limpieza de años
funHist.loc[funHist['anyo'] < 1900, 'anyo'] = 0
funHist.loc[funHist['anyo'] > 2021, 'anyo'] = 0

#Limpieza de descripciones
funHist['descripcion_funcion'] = funHist['descripcion_funcion'].str.title()

#Limpieza de nombres
funHist['nombre'] = funHist['nombre'].str.title()

#Limpieza de Organismo
funHist['organismo_nombre'] = funHist['organismo_nombre'].str.title()

#Limpieza de región
funHist['region'] = funHist['region'].str.title()

#Limpieza de tipo de calificación
funHist['tipo_calificacionp'] = funHist['tipo_calificacionp'].str.title()   

In [26]:
# Eliminamos campos que no entregan valor para trabajar con un dataframe más liviano
funHist = funHist.drop(columns = {'diurnas', 'festivas', 'nocturnas'})

# Limpiamos las rentas que no están en pesos
funHist.loc[(funHist['Tipo Unidad monetaria']!='Pesos') & (funHist['remuneracionbruta_mensual'] > 30000) , 'Tipo Unidad monetaria'] = 'Pesos'


## ***Pendiente limpieza a los grados en super complejo de abordar

#### Descarga de datos para realizar transformaciones de moneda y correción monentaria

In [27]:
#Imports necesarios
import time #librería necesaria para medir el tiempo de ejecución 
import datetime
import pandas as pd
import numpy as np
from functools import reduce
import urllib.request
import json

# Creamos los dataframes vacios para almacenar los valores de UTM y Fechas
dolarDiarioDF = pd.DataFrame(columns=['Fecha', 'Dolar'])
CLFDiarioDF = pd.DataFrame(columns=['Fecha', 'CLF'])
euroDiarioDF = pd.DataFrame(columns=['Fecha', 'Euro'])
utmDF = pd.DataFrame(columns=['Fecha', 'UTM'])
ipcDF = pd.DataFrame(columns=['Fecha', 'IPC'])

# Generamos el ciclo para recorrer los años y leer desde la API los datos de UTM
for i in range(2008, 2021):
    with urllib.request.urlopen("https://mindicador.cl/api/dolar/"+str(i)) as url:
        dolar = json.loads(url.read().decode())
    
    with urllib.request.urlopen("https://mindicador.cl/api/uf/"+str(i)) as url:
        uf = json.loads(url.read().decode())
    
    with urllib.request.urlopen("https://mindicador.cl/api/euro/"+str(i)) as url:
        euro = json.loads(url.read().decode())
    
    with urllib.request.urlopen("https://mindicador.cl/api/utm/"+str(i)) as url:
        utm = json.loads(url.read().decode())
        
    with urllib.request.urlopen("https://mindicador.cl/api/ipc/"+str(i)) as url:
        ipc = json.loads(url.read().decode())
    
    # Generamos los ciclos en que se añaden los dataframes los valores obtenidos desde la API    
    for x in range(len(dolar['serie'])):
        dolarDiarioDF = dolarDiarioDF.append({'Fecha' : dolar['serie'][x]['fecha'][0:7] , 'Dolar' : dolar['serie'][x]['valor']} , ignore_index=True) 
    
    for x in range(len(uf['serie'])):
        CLFDiarioDF = CLFDiarioDF.append({'Fecha' : uf['serie'][x]['fecha'][0:7] , 'CLF' : uf['serie'][x]['valor']} , ignore_index=True)
    
    for x in range(len(euro['serie'])):
        euroDiarioDF = euroDiarioDF.append({'Fecha' : euro['serie'][x]['fecha'][0:7] , 'Euro' : euro['serie'][x]['valor']} , ignore_index=True)
    
    for x in range(len(utm['serie'])):
        utmDF = utmDF.append({'Fecha' : utm['serie'][x]['fecha'][0:7] , 'UTM' : utm['serie'][x]['valor']} , ignore_index=True)
        
    for x in range(len(ipc['serie'])):
        ipcDF = ipcDF.append({'Fecha' : ipc['serie'][x]['fecha'][0:7] , 'IPC' : ipc['serie'][x]['valor']} , ignore_index=True)


# Agrupamos por promedio para efectos prácticos
dolarDF = dolarDiarioDF.groupby(['Fecha'])[['Dolar']].mean()
CLFDF = CLFDiarioDF.groupby(['Fecha'])[['CLF']].mean()
euroDF = euroDiarioDF.groupby(['Fecha'])[['Euro']].mean()

# Unimos los dataframes y lo guardamos en uno nuevo
monedas = reduce(lambda x,y: pd.merge(x,y, on='Fecha', how='left'), [dolarDF, CLFDF, euroDF, utmDF, ipcDF])

# calculamos el IPC en términos de su valor en índice
monedas['IPC'] = monedas['IPC'].fillna(0)
monedas['ipcIndices'] = 0.0
for index, row in monedas.iterrows():
    if index == 0:
        monedas['ipcIndices'][index] =  72.0007284469403   
    else:
        monedas['ipcIndices'][index] = monedas['ipcIndices'][index - 1] + monedas['IPC'][index] * monedas['ipcIndices'][index - 1] / 100

# en base al valor índice calculamos los factores de corrección y limpiamos el dataframe
monedas['factorIPC'] = 0.0
for index, row in monedas.iterrows():
    monedas['factorIPC'][index] = monedas['ipcIndices'][len(monedas)-1] / monedas['ipcIndices'][index]
    
monedas = monedas.drop(columns = ['IPC'])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [28]:
#elimación de datos de IPC innecesarios
monedas = monedas.drop(columns = {'ipcIndices'})

In [ ]:
funHist

In [29]:
#creación del campo mes como "numero" para hacer merge con monedas
funHist.loc[funHist['Mes']=='Enero', 'mesN'] = '01'
funHist.loc[funHist['Mes']=='Febrero', 'mesN'] = '02'
funHist.loc[funHist['Mes']=='Marzo', 'mesN'] = '03'
funHist.loc[funHist['Mes']=='Abril', 'mesN'] = '04'
funHist.loc[funHist['Mes']=='Mayo', 'mesN'] = '05'
funHist.loc[funHist['Mes']=='Junio', 'mesN'] = '06'
funHist.loc[funHist['Mes']=='Julio', 'mesN'] = '07'
funHist.loc[funHist['Mes']=='Agosto', 'mesN'] = '08'
funHist.loc[funHist['Mes']=='Septiembre', 'mesN'] = '09'
funHist.loc[funHist['Mes']=='Octubre', 'mesN'] = '10'
funHist.loc[funHist['Mes']=='Noviembre', 'mesN'] = '11'
funHist.loc[funHist['Mes']=='Diciembre', 'mesN'] = '12'

In [30]:
#creación del campo año mes para fusionar con monedas
funHist['Fecha'] = funHist['anyo'].astype(str) + '-' + funHist['mesN']

In [31]:
# merge con monedas
funHist = funHist.merge(monedas, how = 'left', left_on = 'Fecha', right_on = 'Fecha')

In [33]:
# Calculo de remuneración bruta mensual en pesos reales
funHist['RemBrutaPesosReal'] = 0
funHist.loc[funHist['Tipo Unidad monetaria']=='Pesos', 'RemBrutaPesosReal'] = funHist['remuneracionbruta_mensual'] * funHist['factorIPC']
funHist.loc[funHist['Tipo Unidad monetaria']=='Dólares', 'RemBrutaPesosReal'] = funHist['remuneracionbruta_mensual'] * funHist['factorIPC'] * funHist['Dolar']
funHist.loc[funHist['Tipo Unidad monetaria']=='UTM', 'RemBrutaPesosReal'] = funHist['remuneracionbruta_mensual'] * funHist['factorIPC'] * funHist['UTM']
funHist.loc[funHist['Tipo Unidad monetaria']=='Miles de pesos', 'RemBrutaPesosReal'] = funHist['remuneracionbruta_mensual'] * funHist['factorIPC'] * 1000
funHist.loc[funHist['Tipo Unidad monetaria']=='Euros', 'RemBrutaPesosReal'] = funHist['remuneracionbruta_mensual'] * funHist['factorIPC'] * funHist['Euro']
funHist.loc[funHist['Tipo Unidad monetaria']=='UF', 'RemBrutaPesosReal'] = funHist['remuneracionbruta_mensual'] * funHist['factorIPC'] * funHist['CLF']


In [34]:
# Calculo de montos de horas extrasmensual en pesos reales
funHist['HEDiuReal'] = funHist['montoHExDiu'] * funHist['factorIPC']
funHist['HENocReal'] = funHist['montoHExNoc'] * funHist['factorIPC']
funHist['HEFesReal'] = funHist['montoHExFes'] * funHist['factorIPC']
funHist['ViaticosReal'] = funHist['viaticos'] * funHist['factorIPC']
funHist['HorasExtrasMontoReal'] = funHist['HEDiuReal'] + funHist['HENocReal'] + funHist['HEFesReal']
funHist['HorasExtrasCantTotal'] = funHist['cantHExDiu'] + funHist['cantHExNoc'] + funHist['cantHExFes']


In [35]:
# Eliminamos las monedas a no ser usadas
funHist = funHist.drop(columns = {'factorIPC', 'Dolar', 'UTM', 'Euro', 'CLF'})

In [ ]:
funHist['Regimen'] = funHist['Regimen'].astype('category')
funHist['Tipo Estamento'] = funHist['Tipo Estamento'].astype('category')
funHist['Tipo Unidad monetaria'] = funHist['Tipo Unidad monetaria'].astype('category')
funHist['Tipo cargo'] = funHist['Tipo cargo'].astype('category')
funHist['asignaciones'] = funHist['asignaciones'].astype('category')
funHist['descripcion_funcion'] = funHist['descripcion_funcion'].astype('category')
funHist['fecha_ingreso'] = pd.to_datetime(funHist['fecha_ingreso'])
funHist['fecha_publicacion'] = pd.to_datetime(funHist['fecha_publicacion'])
funHist['fecha_termino'] = pd.to_datetime(funHist['fecha_termino'])
funHist['grado_eus'] = funHist['grado_eus'].astype('category')
funHist['nombre'] = funHist['nombre'].astype('category')
funHist['organismo_codigo'] = funHist['organismo_codigo'].astype('category')
funHist['organismo_nombre'] = funHist['organismo_nombre'].astype('category')
funHist['pago_mensuales'] = funHist['pago_mensuales'].astype('category')
funHist['region'] = funHist['region'].astype('category')
funHist['tipo_calificacionp'] = funHist['tipo_calificacionp'].astype('category')
funHist['mesN'] = funHist['mesN'].astype('category')
funHist['Fecha'] = funHist['Fecha'].astype('category')
funHist['RemBrutaPesosReal'] = funHist['RemBrutaPesosReal'].astype(float)

In [194]:
funHist2 = funHist.copy()

### Respaldo de lo trabajado en bbdd

In [112]:
import os
os.environ['MYSQL_FUNC_USER'] = 'server'
os.environ['MYSQL_FUNC_PASS'] = 'server'
os.environ['MYSQL_FUNC_HOST'] = '192.168.2.2'
os.environ['MYSQL_FUNC_DB'] = 'funcionarios'

#Importamos las credenciales que usaremos en todo el notebook

user = os.getenv('MYSQL_FUNC_USER')
passwd = os.getenv('MYSQL_FUNC_PASS')
host = os.getenv('MYSQL_FUNC_HOST')
db = os.getenv('MYSQL_FUNC_DB')

### Tabla de Servicios

In [163]:
#Código del Trabajo
Servicios = pd.read_csv('http://www.cplt.cl/transparencia_activa/datoabierto/archivos/Organismos_360.csv', encoding = 'cp1252', sep = ';')
Servicios.to_sql(name='Servicios', con=engine, if_exists = 'replace', index=True, chunksize = 10000)

In [182]:
import sqlalchemy as sa

engine2 = sa.create_engine('vertica+vertica_python://dbadmin:1@192.168.2.2:5433/chc')

In [183]:
funHist2 = funHist2.rename(columns = {'Tipo Estamento':'TipoEstamento', 'Tipo Unidad monetaria':'TipoUnidadMonetaria', 
                                     'Tipo cargo': 'TipoCargo', 'descripcion_funcion':'DescripcionFuncion',
                                     'fecha_ingreso': 'FechaIngreso', 'fecha_publicacion':'FechaPublicacion',
                                     'fecha_termino':'FechaTermino', 'grado_eus':'GradoEus', 'organismo_codigo':'OrganismoCodigo',
                                     'organismo_nombre':'OrganismoNombre', 'pago_mensuales':'PagoMensuales',
                                     'tipo_calificacionp':'TipoCalificacionp', 'remuneracionbruta_mensual':'remuneracionbrutamensual'})

In [186]:
funHist2.to_sql(name='full', con=engine2, if_exists = 'replace', index=False ,chunksize = 60000, schema = 'funcionarios',
               dtype={'Mes': VARCHAR(50),
                      'Regimen': VARCHAR(50),
                      'TipoEstamento': VARCHAR(255),
                      'TipoUnidadMonetaria': VARCHAR(50),
                      'TipoCargo': VARCHAR(255),
                      'asignaciones': VARCHAR(2500),
                      'DescripcionFuncion': VARCHAR(8000),
                      'FechaIngreso': VARCHAR(50),
                      'FechaPublicacion': VARCHAR(50),
                      'FechaTermino': VARCHAR(50),
                      'GradoEus': VARCHAR(255),
                      'nombre': VARCHAR(500),
                      'OrganismoCodigo': VARCHAR(50),
                      'OrganismoNombre': VARCHAR(255),
                      'PagoMensuales': VARCHAR(255),
                      'region': VARCHAR(255),
                      'TipoCalificacionp': VARCHAR(2500),
                      'Fecha': VARCHAR(255),
                      'mesN': VARCHAR(50)})

In [193]:
#Servicios = pd.read_csv('http://www.cplt.cl/transparencia_activa/datoabierto/archivos/Organismos_360.csv', encoding = 'cp1252', sep = ';')
Servicios.to_sql(name='Servicios', con=engine2, if_exists = 'replace', index=False ,chunksize = 1000, schema = 'funcionarios',
               dtype={"Codigo_org": VARCHAR(50), 
                        "Organismo" : VARCHAR(255), 
                        "Codigo_padre" : VARCHAR(255), 
                        "Padre_org" : VARCHAR(255), 
                        "Region" : VARCHAR(255), 
                        "Municipalidad" : VARCHAR(255), 
                        "direccion" : VARCHAR(255), 
                        "telefono" : VARCHAR(255), 
                        "url_organismo" : VARCHAR(255), 
                        "url_orgtransparencia" : VARCHAR(255), 
                        "horario_publico" : VARCHAR(500), 
                        "nombre_encargado" : VARCHAR(255), 
                        "email" : VARCHAR(255), 
                        "rut" : VARCHAR(255), 
                        "tipo_cuenta" : VARCHAR(255), 
                        "banco" : VARCHAR(255), 
                        "url_sai" : VARCHAR(255), 
                        "fax" : VARCHAR(255), 
                        "Ingresa" : VARCHAR(255), 
                        "obligadorecibir_sai" : VARCHAR(255), 
                        "organismo_autonomo" : VARCHAR(255), 
                        "interopera" : VARCHAR(255), 
                        "tiene_ta" : VARCHAR(255), 
                        "fecha_ta" : VARCHAR(255)})